In [41]:
from gensim.models import Word2Vec
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from rake_nltk import Rake
from nltk.corpus import brown, movie_reviews, treebank;
import gensim.downloader as api
import fasttext
from fasttext import util
from sklearn.neighbors import NearestNeighbors

In [7]:
model_en = fasttext.load_model('models/cc.en.25.bin')

In [21]:
products = pd.read_csv('data/Products.csv')

In [9]:
orders.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_last_order,product_id,reordered
0,2539329,1,1,2,8,NaN,196,0
1,2539329,1,1,2,8,NaN,14084,0
2,2539329,1,1,2,8,NaN,12427,0
3,2539329,1,1,2,8,NaN,26088,0
4,2539329,1,1,2,8,NaN,26405,0


In [22]:
products.head()

,Name,BarCode,Price,LoaltyCoeff,Point_id
0,Chocolate Sandwich Cookies,903304753514,21.60,0.99,6191
1,All-Seasons Salt,401309075522,25.90,0.58,6870
2,Robust Golden Unsweetened Oolong Tea,851892036179,8.47,0.66,406
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,445454311328,13.53,0.77,5459
4,Green Chile Anytime Sauce,304753955703,22.73,0.24,4954


In [10]:
def text2vector(text: np.array):
    vectors = []

    for subtext in text:
        vector = model_en.get_word_vector(subtext)
        vectors.append(vector)
    
    return vectors

In [93]:
def recommend_product_change(orders, product_id=196, n_neighbors_amount=10, koeff=0.8):
    product_orders_count = len(orders[orders.product_id == product_id])
    product_name = products[products.index == product_id].Name.values
    product_vector = text2vector(product_name)[0]

    other_products_names = products[products.index != product_id].Name.values
    other_products_vector = text2vector(other_products_names)

    knn = NearestNeighbors(n_neighbors=n_neighbors_amount)
    knn.fit(np.array(other_products_vector));

    product_neighbors_ids = knn.kneighbors(product_vector.reshape(1, -1), return_distance=False)[0]
    product_orders_count = len(orders[orders.product_id == product_id])
    other_products_count_mean = len(orders[orders.product_id.isin(product_neighbors_ids)]) / n_neighbors_amount

    if product_orders_count > other_products_count_mean * koeff:
        product_neighbors_ids = []

    return {
        'products': product_neighbors_ids
    }


In [96]:
recommend_product_change(orders, 177)

{'products': array([13669, 31466, 47334,  4834, 47725, 49075,  5506, 36701, 19138,
         5425])}

In [97]:
recommend_product_change(orders, 196)

{'products': []}